In [1]:
# imports here
import numpy as np
import matplotlib.pyplot as plt
import time
from copy import deepcopy


ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Test Function
def sphere_function(x):
    return sum((x**2 - 2*x - 4) ** 2)
   
    
    


In [ ]:
# Problem to be defined here as a class (or struct)

class problem:
    
    def __init__(self):
    # Cost function
        self.cost_function = sphere_function
    # number of genes (variables) in an individial
        self.number_of_genes = 5

    #  Max gene value
        self.maximum_value = 10
    # Min Gene Value
        self.minimum_value = -10


In [ ]:
#Genetic Algorithm Parameters here as a class or struct
class parameters():
    def __init__(self):
        

# Number of iterations of genetic algorithm
        self.number_of_iterations = 100
# Number of individuals in Population 
        self.number_in_population = 50
    
# Rate of new children pre iteration
        self.child_rate = 1
# Number of children 
        self.number_of_children = self.number_in_population * self.child_rate
# Exploration Rate (Gamma)
        self.explore_rate = 0.2
    
# Mutation Rate (Probability that a given gene will be mutated)
        self.mutation_rate = 0.2
# Mutation Range (Determines the amount of change of an individual gene)
        self.mutation_range = 0.2
#  etc.   laters...


    def to_String(self):
        return ("Population: {}  Child Rate: {}  Explore Rate: {}  Mutation Rate: {}  Mutation Range: {}"
                .format(self.number_in_population, self.child_rate, self.explore_rate, self.mutation_rate, self.mutation_range))



In [ ]:
# Structure for an individual of the population as a class or struct
class individual:
    

# Genotype or Chromosone
    chromosone = None
# Cost for the individual
    cost = 0
# Constructor(s)

    # if problem is passed as parameter to constructor, a random individual is generated
    #   https://docs.python.org/3/glossary.html#term-argument
    def __init__(self, problem = None):
        
        if problem is not None:
            self.chromosone = np.random.uniform(problem.minimum_value,problem.maximum_value,problem.number_of_genes)
            self.cost = problem.cost_function(self.chromosone)
# Crossover with another individual  method
    def crossover(self, other_individual, explore_rate):
        child1 = deepcopy(self)
        child2 = deepcopy(other_individual)
        alpha = np.random.uniform(-explore_rate, 1+explore_rate, child1.chromosone.shape)
        child1.chromosone = alpha*self.chromosone + (1-alpha) * other_individual.chromosone
        child2.chromosone = alpha*other_individual.chromosone + (1-alpha) * self.chromosone
        return child1, child2
        
# Mutate method
    def mutate(self, rate, range_of_change):
        for i in range(len(self.chromosone)):
            if np.random.rand() < rate:
                self.chromosone[i] += np.random.randn() * range_of_change


In [ ]:
def choose_distinct_pair_from(population_size):
    index1 = np.random.randint(population_size)
    index2 = np.random.randint(population_size)
    if index1 == index2:
        return choose_distinct_pair_from(population_size)
    else:
        return index1, index2

In [ ]:
# Main Genetic method

def run_genetic (problem, parameters, max_error = 0):
    
    #  read problem
    cost_function = problem.cost_function
    
    # read parameters
    number_in_population = parameters.number_in_population
    number_of_iterations = parameters.number_of_iterations
    number_of_children = parameters.number_of_children
    explore_rate = parameters.explore_rate
    mutation_rate = parameters.mutation_rate
    mutation_range = parameters.mutation_range
    
    
    # placeholder for best solution
    best_solution = deepcopy(individual())
    best_solution.cost = np.infty
    
    # placeholder for best cost at each iteration
    #best_costs = []
    
    population = []
    #  initialize population for the above problem, i.e. generate random individuals 
    for i in range(number_in_population):
        new_individual = individual(problem)
        population.append(new_individual)
        if new_individual.cost < best_solution.cost:
            best_solution = deepcopy(new_individual)
        
    
     # Main Loop for algorithm   (number of iterations)
    
    for iteration in range(number_of_iterations):
        
        # generate a new population of children
        children = []
        
        # How many children? 
        while len(children) < number_of_children:
                
                # Select 2 Parents
                parent1_index, parent2_index = choose_distinct_pair_from(number_in_population)
                parent1 = population[parent1_index]
                parent2 = population[parent2_index]
                
                # Use crossover to produce 2 children
                child1, child2 = parent1.crossover(parent2, explore_rate)
                # Mutate these children
                child1.mutate(mutation_rate, mutation_range)
                child2.mutate(mutation_rate, mutation_range)
                # calculate costs for these children
                child1.cost = cost_function(child1.chromosone)
                child2.cost = cost_function(child2.chromosone)
                
                # add to the children population of new children
                children.append(child1)
                children.append(child2)
                
        # Merge parent and child population
        population += children
        
        # Sort into ascending order for cost
        population = sorted(population, key = lambda x: x.cost)
        
        # Select population next iteration
        population = population[0:number_in_population]
        
        # Update best solution
        if population[0].cost < best_solution.cost:
            best_solution = deepcopy(population[0])
        
        # print iteration results
        #print("Iteration {} Cost {}".format(iteration, best_solution.cost))
        
        #output results ?? class/struct/ best solution/ population?
        #best_costs.append(best_solution.cost)
        
        if best_solution.cost < max_error:
            #print("Stopped at iteration: {}".format(iteration))
            break
                
    #plt.plot(best_costs)
    #plt.semilogy(best_costs)
    #plt.xlabel("Iteration")
    #plt.ylabel("Best Cost")
    #plt.title("Genetic Algorithm \n {}".format(parameters.to_String()))
    return population, best_solution
        
    
    
    
    
    
    

In [ ]:
prob = problem()

In [ ]:
pars = parameters()

In [ ]:
final_pop, best_solution = run_genetic(prob, pars)

In [ ]:
def test_genetic(number_of_runs, problem, parameters, max_error):
    start_time = time.perf_counter()
    for i in range(number_of_runs):
        run_genetic(problem, parameters, max_error)
    return(time.perf_counter() - start_time)

In [ ]:
def test_explore(range_start, range_end, explore_range = 1, range_steps = 1):
    times_for_explore_rate = []
    explore_rates = []
    for i in range(range_start, range_end):
        explore_rate = float(i)/float(explore_range)
        print("Start: {}".format(explore_rate))
        pars.explore_rate = explore_rate
        explore_rates.append(pars.explore_rate)
        times_for_explore_rate.append(test_genetic(100, prob, pars, 0.00001))
        print("Time: {}".format(times_for_explore_rate[i-range_start]))
        print()
    plt.plot(explore_rates, times_for_explore_rate)
    plt.xlabel("Explore rate")
    plt.ylabel("Time in seconds")

# First Test gets range of explore rate from 0 to 100 in steps of 10 (shows 10 to be quickest)

In [ ]:
first_test = test_explore(0,10, 0.1)

# Second test gets explore rate from 0 to 20 in steps of 2 (shows 13 to be quickest)

In [ ]:
second_test = test_explore(0, 21, 1, 2)

# Third test gets explore rate from 12.5 to 13.5 in steps of 0.1

In [ ]:
third_test = test_explore(125,135, 10)